In [17]:
import pandas as pd
import numpy as np

import seaborn as sns

In [2]:
book_df=pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')
rating_df=pd.read_csv('/kaggle/input/book-recommendation-dataset/Ratings.csv')
user_df=pd.read_csv('/kaggle/input/book-recommendation-dataset/Users.csv')

/tmp/ipykernel_32/1373560247.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df=pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')


In [3]:
book_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
rating_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
user_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
print(book_df.shape)
print(rating_df.shape)
print(user_df.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [9]:
book_df.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271358,271360,271358,271360,271360,271357
unique,271360,242135,102022,202,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/185326119X.0...,http://images.amazon.com/images/P/225307649X.0...
freq,1,27,632,13903,7535,2,2,2


In [10]:
# 2 missing value in book records.
#we can either drop these records or fill na with the mode.
book_df.isnull().sum()
book_df.drop(columns=['Book-Author','Publisher'], inplace=True)

In [11]:
rating_df.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [12]:
# No missing value
rating_df.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [13]:
# Total user records: 278858, 40% of age data is missing in User Records

user_df.isnull().sum()
# print(110762/278858)

User-ID          0
Location         0
Age         110762
dtype: int64

## Popularity Based Recommendation System

In [33]:
books=book_df[['ISBN', 'Book-Title']]
ratings=rating_df[['ISBN', 'Book-Rating']]
books_rating_df=books.merge(ratings, on='ISBN')
books_rating_df.head(2)

,ISBN,Book-Title,Book-Rating
0,0195153448,Classical Mythology,0
1,0002005018,Clara Callan,5


In [41]:
agg_books_rating_df=books_rating_df.groupby('Book-Title')['Book-Rating'].agg({'count','mean'}).reset_index()
agg_books_rating_df.rename(columns={'count':'Num-Ratings', 'mean':'Avg-Rating'},inplace=True)
agg_books_rating_df.head()

,Book-Title,Num-Ratings,Avg-Rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00
2,Apple Magic (The Collector's series),1,0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.00
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.00


In [43]:
filtered_books_rating_df=agg_books_rating_df[agg_books_rating_df['Num-Ratings']>=200]
filtered_books_rating_df=filtered_books_rating_df.sort_values(ascending=False,by=['Avg-Rating'])
filtered_books_rating_df.head(20)

,Book-Title,Num-Ratings,Avg-Rating
80434,Harry Potter and the Prisoner of Azkaban (Book 3),428,5.852804
80422,Harry Potter and the Goblet of Fire (Book 4),387,5.824289
80441,Harry Potter and the Sorcerer's Stone (Book 1),278,5.737410
80426,Harry Potter and the Order of the Phoenix (Boo...,347,5.501441
60582,Ender's Game (Ender Wiggins Saga (Paperback)),249,5.409639
80414,Harry Potter and the Chamber of Secrets (Book 2),556,5.183453
191612,The Hobbit : The Enchanting Prelude to The Lor...,281,5.007117
187377,The Fellowship of the Ring (The Lord of the Ri...,368,4.948370
80445,Harry Potter and the Sorcerer's Stone (Harry P...,575,4.895652
211384,"The Two Towers (The Lord of the Rings, Part 2)",260,4.880769
